In [1]:
import csv
import zipfile

In [2]:
import gzip

from collections import defaultdict

In [3]:
import os
os.chdir(r"C:\Users\cdron\Desktop\Cleanup\Misc\kaggle\Facebook_rec_4")
os.getcwd()

'C:\\Users\\cdron\\Desktop\\Cleanup\\Misc\\kaggle\\Facebook_rec_4'

In [4]:
MERCHANDISE_TYPES = ('auto parts', 'books and music', 'clothing', 'computers',
                     'furniture', 'home goods', 'jewelry', 'mobile',
                     'office equipment', 'sporting goods')

In [5]:
COUNTRIES = ('ad', 'ae', 'af', 'ag', 'al', 'am', 'an', 'ao', 'ar', 'at', 'au',
             'aw', 'az', 'ba', 'bb', 'bd', 'be', 'bf', 'bg', 'bh', 'bi', 'bj',
             'bm', 'bn', 'bo', 'br', 'bs', 'bt', 'bw', 'by', 'bz', 'ca', 'cd',
             'cf', 'cg', 'ch', 'ci', 'cl', 'cm', 'cn', 'co', 'cr', 'cv', 'cy',
             'cz', 'de', 'dj', 'dk', 'dm', 'do', 'dz', 'ec', 'ee', 'eg', 'er',
             'es', 'et', 'eu', 'fi', 'fj', 'fo', 'fr', 'ga', 'gb', 'ge', 'gh',
             'gi', 'gl', 'gm', 'gn', 'gp', 'gq', 'gr', 'gt', 'gu', 'gy', 'hk',
             'hn', 'hr', 'ht', 'hu', 'id', 'ie', 'il', 'in', 'iq', 'ir', 'is',
             'it', 'je', 'jm', 'jo', 'jp', 'ke', 'kg', 'kh', 'kr', 'kw', 'kz',
             'la', 'lb', 'li', 'lk', 'lr', 'ls', 'lt', 'lu', 'lv', 'ly', 'ma',
             'mc', 'md', 'me', 'mg', 'mh', 'mk', 'ml', 'mm', 'mn', 'mo', 'mp',
             'mr', 'mt', 'mu', 'mv', 'mw', 'mx', 'my', 'mz', 'na', 'nc', 'ne',
             'ng', 'ni', 'nl', 'no', 'np', 'nz', 'om', 'pa', 'pe', 'pf', 'pg',
             'ph', 'pk', 'pl', 'pr', 'ps', 'pt', 'py', 'qa', 're', 'ro', 'rs',
             'ru', 'rw', 'sa', 'sb', 'sc', 'sd', 'se', 'sg', 'si', 'sk', 'sl',
             'sn', 'so', 'sr', 'sv', 'sy', 'sz', 'tc', 'td', 'tg', 'th', 'tj',
             'tl', 'tm', 'tn', 'tr', 'tt', 'tw', 'tz', 'ua', 'ug', 'uk', 'us',
             'uy', 'uz', 'vc', 've', 'vi', 'vn', 'ws', 'ye', 'za', 'zm', 'zw',
             'zz')

In [6]:
def feature_extraction():
    bidder_dict = defaultdict(int)
    bid_auc_dict = defaultdict(dict)
    bid_merch_dict = defaultdict(dict)
    bid_device_dict = defaultdict(set)
    bid_country_dict = defaultdict(dict)
    bid_ip_dict = defaultdict(set)
    bid_url_dict = defaultdict(set)
    with open('bids.csv', 'rt') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        labels = next(csv_reader)
        for idx, row_ in enumerate(csv_reader):
            if idx % 100000 == 0:
                print ('processed %d' % idx)
            row = dict(zip(labels, row_))
            bid = row['bidder_id']
            bidder_dict[bid] += 1
            if row['auction'] not in bid_auc_dict[bid]:
                bid_auc_dict[bid][row['auction']] = 0
            bid_auc_dict[bid][row['auction']] += 1
            if row['merchandise'] not in bid_merch_dict[bid]:
                bid_merch_dict[bid][row['merchandise']] = 0
            bid_merch_dict[bid][row['merchandise']] += 1
            bid_device_dict[bid].add(row['device'])
            if row['country'] not in bid_country_dict[bid]:
                bid_country_dict[bid][row['country']] = 0
            bid_country_dict[bid][row['country']] += 1
            bid_ip_dict[bid].add(row['ip'])
            bid_url_dict[bid].add(row['url'])
    labels_to_write = ['bidder_id', 'n_auctions', 'n_per_auc', 'n_devices', 
                       'country', 'n_countries', 'n_ip', 'n_url']
    for merch in MERCHANDISE_TYPES:
        labels_to_write.append(merch)
    with open('bid_reduced.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(labels_to_write)
        for bidder in sorted(bidder_dict.keys()):
            row_dict = {k: None for k in labels_to_write}
            row_dict['bidder_id'] = bidder
            row_dict['n_auctions'] = len(bid_auc_dict[bidder])
            row_dict['n_per_auc'] = sum(bid_auc_dict[bidder].values())\
                                    / float(len(bid_auc_dict[bidder]))
            row_dict['n_devices'] = len(bid_device_dict[bidder])
            row_dict['country'] = sorted(bid_country_dict[bidder].items(), 
                                         key=lambda x: x[1],
                                         reverse=True)[0][0]
            row_dict['n_countries'] = len(bid_country_dict[bidder])
            row_dict['n_ip'] = len(bid_ip_dict[bidder])
            row_dict['n_url'] = len(bid_url_dict[bidder])
            for merch in MERCHANDISE_TYPES:
                if merch in bid_merch_dict[bidder]:
                    row_dict[merch] = bid_merch_dict[bidder][merch]
                else:
                    row_dict[merch] = 0
            row_val = [row_dict[col] for col in labels_to_write]
            csv_writer.writerow(row_val)
    return

In [7]:
if __name__ == '__main__':
    feature_extraction()

processed 0
processed 100000
processed 200000
processed 300000
processed 400000
processed 500000
processed 600000
processed 700000
processed 800000
processed 900000
processed 1000000
processed 1100000
processed 1200000
processed 1300000
processed 1400000
processed 1500000
processed 1600000
processed 1700000
processed 1800000
processed 1900000
processed 2000000
processed 2100000
processed 2200000
processed 2300000
processed 2400000
processed 2500000
processed 2600000
processed 2700000
processed 2800000
processed 2900000
processed 3000000
processed 3100000
processed 3200000
processed 3300000
processed 3400000
processed 3500000
processed 3600000
processed 3700000
processed 3800000
processed 3900000
processed 4000000
processed 4100000
processed 4200000
processed 4300000
processed 4400000
processed 4500000
processed 4600000
processed 4700000
processed 4800000
processed 4900000
processed 5000000
processed 5100000
processed 5200000
processed 5300000
processed 5400000
processed 5500000
process

In [8]:
import pandas as pd

In [12]:
from sklearn.grid_search import GridSearchCV

In [9]:
def clean_data(indf):
    for col in ['outcome', 'n_auctions', 'n_devices', 'n_countries', 'n_ip', 
                'n_url'] + list(MERCHANDISE_TYPES):
        if col in indf.columns:
            indf[col] = indf[col].astype(int)
    
    indf['country'] = indf['country'].map({k: i for (i, k)
                                          in enumerate(COUNTRIES)})
    indf.loc[:, 'country'][indf['country'].isnull()] = -1
    indf['country'] = indf['country'].astype(int)

    indf = indf.drop(labels=['payment_account', 'address'], 
                     axis=1)
    return indf

In [10]:
def load_data(do_plots=False):
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    submit_df = pd.read_csv('sampleSubmission.csv')
    bid_df = pd.read_csv('bid_reduced.csv')

    train_df = train_df.merge(bid_df, on='bidder_id', how='inner')
    test_df = test_df.merge(bid_df, on='bidder_id', how='inner')
    
    train_df = clean_data(train_df)
    test_df = clean_data(test_df)
    
    if do_plots:
        from plot_data import plot_data
        plot_data(train_df, prefix='html_train')
        plot_data(test_df, prefix='html_test')

    print (train_df.dtypes)
    print (test_df.dtypes)
    print (submit_df.dtypes)

    xtrain = train_df.drop(labels=['outcome','bidder_id'], axis=1).values
    ytrain = train_df['outcome'].values
    xtest = test_df.drop(labels=['bidder_id'], axis=1).values
    ytest = submit_df
    y_id = list(test_df['bidder_id'])

    return xtrain, ytrain, xtest, ytest, y_id

In [11]:
if __name__ == '__main__':
    xtrain, ytrain, xtest, ytest, y_id = load_data(do_plots=False)

    print ([x.shape for x in (xtrain, ytrain, xtest, ytest)])

bidder_id            object
outcome               int32
n_auctions            int32
n_per_auc           float64
n_devices             int32
country               int32
n_countries           int32
n_ip                  int32
n_url                 int32
auto parts            int32
books and music       int32
clothing              int32
computers             int32
furniture             int32
home goods            int32
jewelry               int32
mobile                int32
office equipment      int32
sporting goods        int32
dtype: object
bidder_id            object
n_auctions            int32
n_per_auc           float64
n_devices             int32
country               int32
n_countries           int32
n_ip                  int32
n_url                 int32
auto parts            int32
books and music       int32
clothing              int32
computers             int32
furniture             int32
home goods            int32
jewelry               int32
mobile                int32
office

C:\Anaconda3 2.1.0\lib\site-packages\IPython\kernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.cross_validation import train_test_split

from sklearn.metrics import roc_auc_score

In [17]:
from sklearn import svm

In [19]:
def test_model(model, xtrain, ytrain):
    xTrain, xTest, yTrain, yTest = train_test_split(xtrain, ytrain,
                                                    test_size=0.2)
    model.fit(xTrain, yTrain)
    
    
    yprob = model.predict_proba(xTest)
    
    print ('score', roc_auc_score(yTest, yprob[:,1]))
    return

In [26]:
def prepare_submission(model, xtest, ytest, yid):
    yprob = model.predict_proba(xtest)
    print (yprob.shape, ytest.shape)
    for idx, row in ytest.iterrows():
        if row['bidder_id'] in yid:
            idy = yid.index(row['bidder_id'])
            ytest.loc[idx, 'prediction'] = yprob[idy, 1]
        else:
            ytest.loc[idx, 'prediction'] = 0.05 # pure hack
    print (ytest.shape)
    ytest.to_csv('may6th_submission_2.csv', index=False)
    return

In [25]:
if __name__ == '__main__':
    xtrain, ytrain, xtest, ytest, yid = load_data(do_plots=False)

#    model = LogisticRegression()
#    model = RandomForestClassifier()
    model = GradientBoostingClassifier(n_estimators=200, min_samples_split=3, max_depth=9, verbose=1)
#  model = svm.NuSVC(probability=True)
    test_model(model, xtrain, ytrain)

    model.fit(xtrain, ytrain)
    
    prepare_submission(model, xtest, ytest, yid)

bidder_id            object
outcome               int32
n_auctions            int32
n_per_auc           float64
n_devices             int32
country               int32
n_countries           int32
n_ip                  int32
n_url                 int32
auto parts            int32
books and music       int32
clothing              int32
computers             int32
furniture             int32
home goods            int32
jewelry               int32
mobile                int32
office equipment      int32
sporting goods        int32
dtype: object
bidder_id            object
n_auctions            int32
n_per_auc           float64
n_devices             int32
country               int32
n_countries           int32
n_ip                  int32
n_url                 int32
auto parts            int32
books and music       int32
clothing              int32
computers             int32
furniture             int32
home goods            int32
jewelry               int32
mobile                int32
office

C:\Anaconda3 2.1.0\lib\site-packages\IPython\kernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
model

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=9, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              random_state=None, subsample=1.0, verbose=1,
              warm_start=False)